In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Thesis")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

23/01/19 22:00:26 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
spark.stop()

In [1]:
spark

In [10]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.2 MB/s 
     |████████████████████████████████| 7.6 MB 41.4 MB/s 
     |████████████████████████████████| 182 kB 65.0 MB/s 


In [11]:
# thu vien chinh sua cac loi unicode
!pip install ftfy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.2 MB/s 


In [12]:
import pandas as pd
import numpy as np
import os
from pprint import pprint
import string    
import random
import json
import spacy
from spacy import displacy
from transformers import BertTokenizer, BertForTokenClassification
from transformers import TFRobertaModel
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-12-07 17:11:07--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-12-07 17:11:08--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-12-07 17:11:08--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [6]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

import json
import os
import shutil

In [7]:
spark = sparknlp.start()

In [9]:

#save_model_address = '/content/drive/MyDrive/LUẬN VĂN-K18_CQ/02. Bình-Ngọc/Data/Dataset/conll/b_simp'
#save_model_address = '/content'

save_model_address = '/content/drive/MyDrive/Data Science/thesis/ML_NER/NERModel_config'

#save_model = BertForTokenClassification.from_pretrained(save_model_address, num_labels=20)
#tokenizer = BertTokenizer.from_pretrained(save_model_address,do_lower_case=True)

model = AutoModelForTokenClassification.from_pretrained(save_model_address)
tokenizer = AutoTokenizer.from_pretrained(save_model_address, do_lower_case=True, model_max_length=256)

#nlp = pipeline("ner", model=save_model, tokenizer=tokenizer, aggregation_strategy='simple',ignore_labels =['X','O'])


In [ ]:
loaded_ner_model = NerDLModel.load("/content/drive/MyDrive/Data Science/thesis/ML_NER/NERModel_config/pytorch_model")

In [59]:
df = spark.read.option('header','true').csv('/home/tyeson/Desktop/K18/File K18/sample_crawl_dataset/Dice_12JobTitle.csv',inferSchema=True)
#df = spark.read.option('header','true').csv('/content/drive/MyDrive/Data Science/thesis/K18/File K18/sample_crawl_dataset/Coursera_DataScience.csv',inferSchema=True)

In [60]:
df.show(10)

+--------------------+--------------------+--------------------+
|                name|         description|              salary|
+--------------------+--------------------+--------------------+
|Full Stack Developer| Job Description....| $110,000 - $135,000|
|Full Stack Developer| Job Description....|   $85,000 - $95,000|
|Full Stack Developer| Job Description....|    No Salary Listed|
|Sr. Full Stack De...|" Job Description...| Deloitte's Gover...|
|.NET Full Stack D...| Job Description....|    No Salary Listed|
|Full Stack Developer| Job Description....|    No Salary Listed|
|Applied AI Full S...| Job Description....|    No Salary Listed|
|Full Stack Developer|" Job Description...| ideally at an en...|
|Artificial Intell...| Job Description....|    No Salary Listed|
|Artificial Intell...| Job Description....|    No Salary Listed|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [61]:
import pyspark.sql.functions as f
from pyspark.sql.functions import trim,ltrim,rtrim
from pyspark.sql.functions import split, col, monotonically_increasing_id
from pyspark.sql.functions import lower, col
df = df.withColumn("id", monotonically_increasing_id()).withColumn('name', lower(col('name')))
df.show(1000)

+--------------------+--------------------+--------------------+---+
|                name|         description|              salary| id|
+--------------------+--------------------+--------------------+---+
|full stack developer| Job Description....| $110,000 - $135,000|  0|
|full stack developer| Job Description....|   $85,000 - $95,000|  1|
|full stack developer| Job Description....|    No Salary Listed|  2|
|sr. full stack de...|" Job Description...| Deloitte's Gover...|  3|
|.net full stack d...| Job Description....|    No Salary Listed|  4|
|full stack developer| Job Description....|    No Salary Listed|  5|
|applied ai full s...| Job Description....|    No Salary Listed|  6|
|full stack developer|" Job Description...| ideally at an en...|  7|
|artificial intell...| Job Description....|    No Salary Listed|  8|
|artificial intell...| Job Description....|    No Salary Listed|  9|
|sr. full stack de...| Job Description....|Depends on Experi...| 10|
|full stack developer| Job Descrip

In [62]:
df = df.withColumn("name", f.regexp_replace(f.col("name"), "senior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "senior', ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr.", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), ", senior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr. ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "jr.", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "mid-senior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "lead", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "with", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "remote", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "onsite", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "mid Level", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "associate", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), ", consultant", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "senior Consultant", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "iii", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "ii", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " , ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " intern ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " intern", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "junior ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), " junior", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "campaign ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "contract: ", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "\(.*?\)", "").alias("name")) \
    .withColumn("name", f.regexp_replace(f.col("name"), "sr ", "").alias("name")) \
    .withColumn("name", trim(col("name"))).alias("name")

df.show(1000)


+--------------------+--------------------+--------------------+---+
|                name|         description|              salary| id|
+--------------------+--------------------+--------------------+---+
|full stack developer| Job Description....| $110,000 - $135,000|  0|
|full stack developer| Job Description....|   $85,000 - $95,000|  1|
|full stack developer| Job Description....|    No Salary Listed|  2|
|full stack develo...|" Job Description...| Deloitte's Gover...|  3|
|.net full stack d...| Job Description....|    No Salary Listed|  4|
|full stack developer| Job Description....|    No Salary Listed|  5|
|applied ai full s...| Job Description....|    No Salary Listed|  6|
|full stack developer|" Job Description...| ideally at an en...|  7|
|artificial intell...| Job Description....|    No Salary Listed|  8|
|artificial intell...| Job Description....|    No Salary Listed|  9|
|full stack developer| Job Description....|Depends on Experi...| 10|
|full stack developer| Job Descrip

In [63]:
career = df.select(df.name)
career = career.dropDuplicates(['name']).select('name')
career = career.withColumn("name", ltrim(col("name"))).alias("name")
career.show()

+--------------------+
|                name|
+--------------------+
|frontend develope...|
|ai software engineer|
|full stack develo...|
|      data scientist|
|data science arch...|
|frontend develope...|
|      unity engineer|
|principal big dat...|
|entry level/sql d...|
|ios/android devel...|
|game systems engi...|
|developer w/ bots...|
|automated testing...|
|java backend deve...|
|database administ...|
|408 artificial in...|
|master big data a...|
| full stack engineer|
|big data developm...|
|snowflake databas...|
+--------------------+
only showing top 20 rows



In [64]:
career.count()

426

In [55]:
career.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("node.keys", "name")\
    .option("labels",":Name")\
    .save()

In [67]:
df = df.withColumn("timestamp", f.current_timestamp()).alias("timestamp")
jobpost = df.select(df.id, df.timestamp)
jobpost.show()

+---+--------------------+
| id|           timestamp|
+---+--------------------+
|  0|2023-01-20 13:20:...|
|  1|2023-01-20 13:20:...|
|  2|2023-01-20 13:20:...|
|  3|2023-01-20 13:20:...|
|  4|2023-01-20 13:20:...|
|  5|2023-01-20 13:20:...|
|  6|2023-01-20 13:20:...|
|  7|2023-01-20 13:20:...|
|  8|2023-01-20 13:20:...|
|  9|2023-01-20 13:20:...|
| 10|2023-01-20 13:20:...|
| 11|2023-01-20 13:20:...|
| 12|2023-01-20 13:20:...|
| 13|2023-01-20 13:20:...|
| 14|2023-01-20 13:20:...|
| 15|2023-01-20 13:20:...|
| 16|2023-01-20 13:20:...|
| 17|2023-01-20 13:20:...|
| 18|2023-01-20 13:20:...|
| 19|2023-01-20 13:20:...|
+---+--------------------+
only showing top 20 rows



In [69]:
jobpost.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("node.keys", "id")\
    .option("labels",":JobPosting")\
    .save()

In [ ]:
df.write.format("org.neo4j.spark.DataSource")\
    .mode("Overwrite")\
    .option("url", "neo4j+s://c4ae1994.databases.neo4j.io")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "_qALa8tQkC7Bg47nvzrRPhuPH38QJEUXeUw1NTs22J8")\
    .option("relationship", 'HAS_LEVEL')\
    .option('relationship.save.strategy','keys')\
    .option('relationship.source.labels',':Career')\
    .option('relationship.source.node.keys','source:Career')\
    .option('relationship.source.save.mode','Match')\
    .option('relationship.target.labels',':JobPosting')\
    .option('relationship.target.node.keys','target:JonPosting')\
    .option('relationship.target.save.mode','Match')\
    .save()